In [1]:
!wget http://www.saifmohammad.com/WebDocs/AIT-2018/AIT2018-DATA/EI-reg/English/EI-reg-En-train.zip
!wget http://saifmohammad.com/WebDocs/AIT-2018/AIT2018-DATA/EI-reg/English/2018-EI-reg-En-dev.zip

--2021-11-19 04:55:01--  http://www.saifmohammad.com/WebDocs/AIT-2018/AIT2018-DATA/EI-reg/English/EI-reg-En-train.zip
Resolving www.saifmohammad.com (www.saifmohammad.com)... 192.185.17.122
Connecting to www.saifmohammad.com (www.saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 356461 (348K) [application/zip]
Saving to: ‘EI-reg-En-train.zip’

EI-reg-En-train.zip 100%[===================>] 348.11K  1.24MB/s    in 0.3s    

2021-11-19 04:55:02 (1.24 MB/s) - ‘EI-reg-En-train.zip’ saved [356461/356461]

--2021-11-19 04:55:02--  http://saifmohammad.com/WebDocs/AIT-2018/AIT2018-DATA/EI-reg/English/2018-EI-reg-En-dev.zip
Resolving saifmohammad.com (saifmohammad.com)... 192.185.17.122
Connecting to saifmohammad.com (saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 83779 (82K) [application/zip]
Saving to: ‘2018-EI-reg-En-dev.zip’

2018-EI-reg-En-dev. 100%[===================>]  81.8

In [2]:
!unzip EI-reg-En-train.zip
!unzip 2018-EI-reg-En-dev.zip

Archive:  EI-reg-En-train.zip
  inflating: EI-reg-En-anger-train.txt  
  inflating: EI-reg-En-fear-train.txt  
  inflating: EI-reg-En-joy-train.txt  
  inflating: EI-reg-En-sadness-train.txt  
Archive:  2018-EI-reg-En-dev.zip
  inflating: 2018-EI-reg-En-anger-dev.txt  
  inflating: 2018-EI-reg-En-fear-dev.txt  
  inflating: 2018-EI-reg-En-joy-dev.txt  
  inflating: 2018-EI-reg-En-sadness-dev.txt  


In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import re

In [4]:
!pip install sentence_transformers

     |████████████████████████████████| 78 kB 3.4 MB/s 
     |████████████████████████████████| 3.1 MB 16.2 MB/s 
     |████████████████████████████████| 3.3 MB 25.1 MB/s 
     |████████████████████████████████| 1.2 MB 51.7 MB/s 
     |████████████████████████████████| 59 kB 6.1 MB/s 
     |████████████████████████████████| 895 kB 72.1 MB/s 
     |████████████████████████████████| 596 kB 53.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=327f1b0555bd18693173d8336fffc4eb1287fcfecb08168bce1e714b8edb5b6d
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
def load_dataset(emotion):
  train_data = pd.read_csv(f"EI-reg-En-{emotion}-train.txt", delimiter='\t')
  test_data = pd.read_csv(f"2018-EI-reg-En-{emotion}-dev.txt", delimiter='\t')
  
  train_sentences = train_data["Tweet"].tolist()
  y_train = train_data["Intensity Score"].to_numpy().reshape((-1, 1))
  
  test_sentences = test_data["Tweet"].tolist()
  y_test = test_data["Intensity Score"].to_numpy().reshape((-1, 1))

  print(f"{emotion} Dataset Loaded")
  return train_sentences, y_train, test_sentences, y_test

In [7]:
def pre_process(tweet):
  tweet = tweet.lower()
  tweet = tweet.encode("ascii", "ignore").decode() # removes emoticons and non-English characters
  tweet = re.sub(r"@{1}[a-z0-9_]+\s", " ", tweet) # removes username mentions
  tweet = re.sub(r"htt(p|ps)\S+", " ", tweet) # removes links in the tweet
  tweet = re.sub(r'[a-z0-9._%-]+@[a-z0-9.-]+\.[a-z]{2,4}', " ", tweet) # removes email
  tweet = re.sub(r"#", "", tweet)
  tweet = re.sub(r"\\n|\\t", " ", tweet)
  tweet = " ".join(tweet.split()) # removing multiple spaces between words
  return tweet

In [8]:
emotions = ["anger", "fear", "joy", "sadness"]

In [9]:
raw_sentences_train = {}
raw_sentences_test = {}
train_intensities = {}
test_intensities = {}

In [10]:
for emotion in emotions:
  data = load_dataset(emotion)
  raw_sentences_train[emotion] = data[0]
  raw_sentences_test[emotion] = data[2]
  train_intensities[emotion] = data[1]
  test_intensities[emotion] = data[3]

anger Dataset Loaded
fear Dataset Loaded
joy Dataset Loaded
sadness Dataset Loaded


In [11]:
train_sentences = {}
for emotion in emotions:
  train_sentences[emotion] = list(map(pre_process, raw_sentences_train[emotion]))

In [12]:
train_embeddings = {}
for emotion in emotions:
  train_embeddings[emotion] = embedding_model.encode(train_sentences[emotion])

In [13]:
class ShallowNN(nn.Module):
  
  def __init__(self, input_dim, hidden_dim, dropout=0):

    super(ShallowNN, self).__init__()
    self.layer1 = nn.Linear(input_dim, hidden_dim)
    self.activation1 = nn.ReLU()
    self.layer2 = nn.Linear(hidden_dim, 1)
  
  def forward(self, X):
    out1 = self.layer1(X)
    out1 = self.activation1(out1)
    output = self.layer2(out1)
    return output

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [15]:
def trainML(model, X, y):
  model.fit(X, y.flatten())
  return model

In [16]:
def trainNN(model, X, y, n_iters=1000, lr=0.03):
  X = torch.tensor(X, dtype=torch.float32)
  y = torch.tensor(y, dtype=torch.float32)

  loss_function = nn.MSELoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=lr)

  for i in range(n_iters):
    optimizer.zero_grad()
    predictions = model(X)
    loss = loss_function(predictions, y)
    loss.backward()
    optimizer.step()
  
  return model

In [17]:
def getModels(train_embeddings, y_train):

  LR_model = trainML(LinearRegression(), train_embeddings, y_train)
  SVR_model = trainML(SVR(), train_embeddings, y_train)
  RFR_model = trainML(RandomForestRegressor(n_estimators=60), train_embeddings, y_train)

  print("ML models trained")

  input_dim = train_embeddings.shape[1]
  hidden_dim = 64
  NN_model = trainNN(ShallowNN(input_dim, hidden_dim), train_embeddings, y_train)

  print("NN model trained")

  return LR_model, SVR_model, RFR_model, NN_model


In [18]:
from scipy.stats import pearsonr

In [19]:
def evaluateMLModel(model, X, y):
  y_pred = model.predict(X)
  return pearsonr(y.flatten(), y_pred)[0]

In [20]:
def evaluateNNModel(model, X, y):
  X = torch.tensor(X, dtype=torch.float32)
  with torch.no_grad():
    y_pred = model(X)
  y_pred = y_pred.numpy().flatten()
  return pearsonr(y.flatten(), y_pred)[0]

In [21]:
LR_models = {}
SVR_models = {}
RFR_models = {}
NN_models = {}

In [26]:
for emotion in emotions:
  print()
  print("Emotion which is used as test set:", emotion)
  print()
  X_train = None
  y_train = None
  for key in emotions:
    if emotion == key:
      continue
    if X_train is None:
      X_train = train_embeddings[key]
      y_train = train_intensities[key]
    else:
      X_train = np.vstack([X_train, train_embeddings[key]])
      y_train = np.vstack([y_train, train_intensities[key]])

  models = getModels(X_train, y_train)
  LR_models[emotion] = models[0]
  SVR_models[emotion] = models[1]
  RFR_models[emotion] = models[2]
  NN_models[emotion] = models[3]


Emotion which is used as test set: anger

ML models trained
NN model trained

Emotion which is used as test set: fear

ML models trained
NN model trained

Emotion which is used as test set: joy

ML models trained
NN model trained

Emotion which is used as test set: sadness

ML models trained
NN model trained


In [27]:
test_sentences = {}
for emotion in emotions:
  test_sentences[emotion] = list(map(pre_process, raw_sentences_test[emotion]))

In [28]:
test_embeddings = {}
for emotion in emotions:
  test_embeddings[emotion] = embedding_model.encode(test_sentences[emotion])

In [29]:
scores = []
for emotion in emotions:
  scores.append([])
  scores[-1].append(evaluateMLModel(LR_models[emotion],
                                    test_embeddings[emotion],
                                    test_intensities[emotion]))
  scores[-1].append(evaluateMLModel(SVR_models[emotion],
                                    test_embeddings[emotion],
                                    test_intensities[emotion]))
  scores[-1].append(evaluateMLModel(RFR_models[emotion],
                                    test_embeddings[emotion],
                                    test_intensities[emotion]))
  scores[-1].append(evaluateNNModel(NN_models[emotion],
                                    test_embeddings[emotion],
                                    test_intensities[emotion]))

In [30]:
headings = ["Linear Regresison", "SVR", "Random Forest", "Shallow NN"]

In [31]:
df = pd.DataFrame(scores, columns=headings, index=emotions)

In [32]:
df

,Linear Regresison,SVR,Random Forest,Shallow NN
anger,0.244427,0.272509,0.394399,0.221281
fear,0.300466,0.349048,0.339295,0.348878
joy,-0.474741,-0.512914,-0.591285,-0.538595
sadness,0.327861,0.366970,0.384038,0.325190
